## creating a n-dimensional hypersphere 

encodes the RGB information of each pixel of an image.

In [97]:
from PIL import Image
from qiskit.quantum_info import Statevector

import numpy as np
from scipy.interpolate import Rbf

coverting image to n-dimensional unit statevectors on an n-dimensional bloch sphere

In [74]:
image_path = "/Users/devaldeliwala/quantum_image_encryption/images/el_primo_square.jpg"
n =8

im = Image.open(image_path, 'r')
image = im.convert("RGB")

n_pixels = np.array(image).shape[0]**2
pixel_values = np.array(list(image.getdata()))

def rgb_to_decimal(r, g, b):
    return (r << 16) + (g << 8) + b

def decimal_to_rgb(decimal):
    r = (decimal >> 16) & 0xFF
    g = (decimal >> 8) & 0xFF
    b = decimal & 0xFF
    return r, g, b

int_values = []
for r,g,b in pixel_values: 
    int_values.append(rgb_to_decimal(r, g, b))
int_values = np.array(int_values)

recovered_rgb_values = []
for i in int_values: 
    recovered_rgb_values.append(decimal_to_rgb(i))
recovered_rgb_values = np.array(recovered_rgb_values)

# checking for reversibility
if(np.allclose(pixel_values, recovered_rgb_values)): 
   print("rgb-int reversible conversion success")

rgb-int reversible conversion success


In [86]:
groups = []

i = 0 
while i < n_pixels: 
    groups.append(int_values[i:i + 2 ** n])
    i += 2**n

def to_unit_vector(arr):
    magnitude = np.linalg.norm(arr)
    if magnitude == 0:
        return arr, magnitude
    unit_vector = arr / magnitude
    return unit_vector, magnitude

def from_unit_vector(unit_vector, magnitude):
    return unit_vector * magnitude

unit_vector_groups = []
magnitudes = []

# unit_vector_groups stores 2^n-dimensional unit vectors 
for i in groups: 
    unit_vector, magnitude = to_unit_vector(i)
    magnitudes.append(magnitude)
    unit_vector_groups.append(unit_vector)

In [136]:
pixel_values[:, 0][0]

19

In [147]:
rbf_r_list = []
rbf_g_list = []
rbf_b_list = []

r_values = [pixel[0] for pixel in pixel_values]
g_values = [pixel[1] for pixel in pixel_values]
b_values = [pixel[2] for pixel in pixel_values]

# Setup RBF for each dimension separately
for i in range(256):
    rbf_r = Rbf(*np.array(unit_vector_groups).T, r_values[i::256], function='multiquadric')
    rbf_g = Rbf(*np.array(unit_vector_groups).T, g_values[i::256], function='multiquadric')
    rbf_b = Rbf(*np.array(unit_vector_groups).T, b_values[i::256], function='multiquadric')
    rbf_r_list.append(rbf_r)
    rbf_g_list.append(rbf_g)
    rbf_b_list.append(rbf_b)

recovered_pixel_values = []
for j in unit_vector_groups: 
    r_values = [rbf_r_list[i](*j) for i in range(256)]
    g_values = [rbf_g_list[i](*j) for i in range(256)]
    b_values = [rbf_b_list[i](*j) for i in range(256)]

    rgb_values = np.array([np.clip([r, g, b], 0, 255).astype(int) for r, g, b in zip(r_values, g_values, b_values)])
    recovered_pixel_values.append(rgb_values)

recovered_pixel_values

[array([[ 19, 102, 234],
        [ 19, 103, 235],
        [ 18, 102, 235],
        [ 17, 101, 234],
        [ 16,  99, 233],
        [ 15, 100, 234],
        [ 12,  99, 236],
        [ 12,  99, 236],
        [ 16, 102, 241],
        [ 16, 101, 241],
        [ 16, 102, 244],
        [ 18, 103, 245],
        [ 18, 103, 247],
        [ 19, 104, 248],
        [ 21, 106, 250],
        [ 21, 106, 251],
        [ 20, 106, 254],
        [ 18, 109, 252],
        [ 16, 111, 250],
        [ 13, 114, 249],
        [ 13, 114, 252],
        [ 17, 113, 252],
        [ 25, 110, 253],
        [ 29, 110, 248],
        [ 39, 117, 253],
        [ 32, 116, 250],
        [ 24, 115, 255],
        [ 17, 114, 255],
        [ 19, 111, 255],
        [ 22, 112, 255],
        [ 25, 115, 255],
        [ 28, 118, 253],
        [ 20, 114, 250],
        [ 22, 117, 253],
        [ 28, 118, 255],
        [ 27, 118, 254],
        [ 23, 115, 255],
        [ 20, 113, 254],
        [ 17, 112, 255],
        [ 14, 114, 255],


In [176]:
final_pixels = []
for i in recovered_pixel_values: 
    for j in i: 
        final_pixels.append(j)
final_pixels = np.array(final_pixels).reshape(64, 64, 3)

In [179]:
image = Image.fromarray(np.uint8(final_pixels), 'RGB')
image.show()

In [92]:
# converting each of the 2^n dimensional unit vectors reversibly into 
# n dimensional statevectors 

def unit_vector_to_statevector(unit_vector):
    dim = len(unit_vector) // 2
    real_parts = unit_vector[:dim]
    imag_parts = unit_vector[dim:]
    statevector = real_parts + 1j * imag_parts
    statevector /= np.linalg.norm(statevector)
    return statevector

def statevector_to_unit_vector(statevector):
    real_parts = np.real(statevector)
    imag_parts = np.imag(statevector)
    combined = np.concatenate([real_parts, imag_parts])
    unit_vector = combined / np.linalg.norm(combined)
    return unit_vector

statevectors = []
for i in unit_vector_groups: 
    statevectors.append(unit_vector_to_statevector(i))

unit_vectors = []
for i in statevectors:
    unit_vectors.append(statevector_to_unit_vector(i))
    
statevectors = [Statevector(i) for i in statevectors]

for i in statevectors:
    display(i.draw('latex'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

converting the n-dimensional statevectors back into the RGB values that encode the image

In [17]:
def statevector_to_unit_vector(statevector):
    real_parts = np.real(statevector)
    imag_parts = np.imag(statevector)
    combined = np.concatenate([real_parts, imag_parts])
    unit_vector = combined / np.linalg.norm(combined)
    return unit_vector

recovered_unit_vectors = []

for i in statevectors:
    recovered_unit_vectors.append(statevector_to_unit_vector(i.data))


int_vectors = []
for i in range(len(recovered_unit_vectors)): 
    int_vectors.append(recovered_unit_vectors[i] * magnitudes[i])

recovered_int_values = []
for i in int_vectors: 
    for j in i: 
        recovered_int_values.append(j)
    
recovered_int_values = np.array(recovered_int_values)

recovered_pixel_values = []
for i in recovered_int_values: 
    recovered_pixel_values.append(decimal_to_rgb(int(i)))
recovered_pixel_values = np.array(recovered_rgb_values)


In [45]:
width, height = image.size

reshaped_pixel_values = recovered_pixel_values.reshape((height, width ,3))
reconstructed_image = Image.fromarray(reshaped_pixel_values.astype(np.uint8), 'RGB')
reconstructed_image.show()

In [49]:
np.array(int_vectors).flatten()

array([1271787., 1271787., 1205996., ...,  679900.,  614107.,  482521.])

In [53]:
recovered_int_values = []
for i in int_vectors:
    for j in i: 
        recovered_int_values.append(j)
recovered_int_values = np.array(recovered_int_values)

In [95]:
a = Statevector([0.0508522 +0.03246801j, 0.0508522 +0.03246801j,
             0.04822156+0.03508854j, 0.04559083+0.04034998j,
             0.04296011+0.0429807j , 0.04296011+0.04561143j,
             0.03508862+0.04824223j, 0.03508862+0.05087295j,
             0.04298086+0.05088331j, 0.04298086+0.05088331j,
             0.04560143+0.05351411j, 0.04823215+0.05614483j,
             0.05086291+0.05877559j, 0.05349364+0.06140632j,
             0.05612444+0.06403704j, 0.05613472+0.05618602j,
             0.05613484+0.04313496j, 0.04829385+0.05097583j,
             0.04308358+0.08498022j, 0.03787336+0.09022111j,
             0.03786324+0.06932919j, 0.04834507+0.05889843j,
             0.06927797+0.06152903j, 0.07974932+0.05629845j,
             0.10340515+0.05371911j, 0.08768239+0.04586801j,
             0.0640883 +0.03541693j, 0.04573494+0.04851908j,
             0.05094512+0.09037429j, 0.05880646+0.10346625j,
             0.06931896+0.08777436j, 0.07459052+0.07731309j,
             0.05621656+0.06416991j, 0.05886784+0.0694212j ,
             0.07460079+0.0799031j , 0.07197011+0.08512352j,
             0.06407806+0.08770303j, 0.05619625+0.08767232j,
             0.04833491+0.08504164j, 0.04048381+0.07980074j,
             0.04311449+0.07455985j, 0.03786332+0.06405759j,
             0.04046326+0.06143707j, 0.04829373+0.0561859j ,
             0.06139584+0.05881662j, 0.06923655+0.05880626j,
             0.0822978 +0.0614062j , 0.0770567 +0.05877555j,
             0.06661574+0.05350407j, 0.04562154+0.04828373j,
             0.03769923+0.03259172j, 0.05342198+0.04567316j,
             0.09534861+0.08226661j, 0.10581996+0.0822461j ,
             0.09268718+0.04563202j, 0.06122127+0.03520189j,
             0.04560147+0.0089153j , 0.01672478+0.05880634j,
             0.03771862+0.07435405j, 0.03504643+0.0927895j ,
             0.09018916+0.03252959j, 0.04019516+0.04315516j,
             0.04546688+0.01415559j, 0.06388194+0.03516031j,
             0.04296003+0.03245781j, 0.04296003+0.03508854j,
             0.04559083+0.03771926j, 0.04559083+0.0429807j ,
             0.04559083+0.04561143j, 0.04559087+0.05087295j,
             0.04035006+0.05350367j, 0.04298078+0.05352419j,
             0.04824231+0.05614475j, 0.04824231+0.05614475j,
             0.05086287+0.05877555j, 0.0534936 +0.05617562j,
             0.05612436+0.06403704j, 0.05875508+0.06142687j,
             0.06138588+0.06929848j, 0.05876544+0.06144738j,
             0.05093489+0.05101678j, 0.05093489+0.0640883j ,
             0.05879623+0.09810292j, 0.05881666+0.10596426j,
             0.04835535+0.07198035j, 0.05360647+0.05891898j,
             0.06669843+0.05893945j, 0.07454945+0.05632928j,
             0.08509278+0.04586801j, 0.07461103+0.04327827j,
             0.05366793+0.0380782j , 0.05102701+0.05643128j,
             0.06671898+0.10352743j, 0.07458032+0.11398874j,
             0.07722124+0.09305624j, 0.07725187+0.08258481j,
             0.05887792+0.07206192j, 0.06413944+0.07730293j,
             0.0746315 +0.08253367j, 0.07724171+0.08776432j,
             0.07194964+0.09034383j, 0.06668827+0.09294384j,
             0.06144738+0.09292337j, 0.05882694+0.09029272j,
             0.05359628+0.08767232j, 0.04833491+0.07981098j,
             0.04571435+0.06669851j, 0.05092445+0.06407806j,
             0.0614062 +0.06146785j, 0.06924686+0.06146785j,
             0.07445701+0.05883717j, 0.07183648+0.05357573j,
             0.06135466+0.05879623j, 0.04826274+0.05617578j,
             0.0377608 +0.04309398j, 0.05346285+0.04569367j,
             0.09010759+0.07182605j, 0.09795858+0.06921584j,
             0.07700556+0.03257109j, 0.05344218+0.03521193j,
             0.0088125 +0.06658471j, 0.0377604 +0.10582931j,
             0.05607218+0.11352596j, 0.05867187+0.12154168j,
             0.05344138+0.05094464j, 0.05347225+0.03003273j,
             0.04817993+0.02198622j, 0.06133354+0.04821112j],
            dims=(2, 2, 2, 2, 2, 2, 2))

In [96]:
a.draw('latex')

<IPython.core.display.Latex object>